In [4]:
import os
import random

from numpy.random import choice
import faker
import pandas as pd
from tqdm.notebook import trange

The dataset is available in the [Google Drive](https://drive.google.com/file/d/1gviMipwq5MM_0O_gE02PsTq0-2tNGfbo/view?usp=sharing). Download it into the `data` folder.

In [27]:
n = 10_000_000
noize = 0.003
nf = round(1 / noize)

dr = f'data'
f = f'fake_data_people_{n}_{noize}.parquet'

filename = os.path.join(dr, f)
if f not in os.listdir(dr):
    fake = faker.Faker(seed=0)
    data = []
    for i in trange(n):
        # Faker produce a lot of duplicates, so we need to add some noise
        rnd_int = random.randint(1, nf)
        rnd_int_str = str(rnd_int)
        rnd_int_for_email = str(rnd_int // 2)
        data.append({
            'name': fake.name() + rnd_int_str,
            'email': fake.email() + rnd_int_for_email,
            'address': fake.address()
        })
    df = pd.DataFrame(data)
    df.to_parquet(filename, index=False)

df = pd.read_parquet(filename)
print(f"% of duplicate names: {df['name'].value_counts().loc[lambda x: x > 1].shape[0] / df.shape[0] * 100:.2f}%")
print(f"% of duplicate emails: {df['email'].value_counts().loc[lambda x: x > 1].shape[0] / df.shape[0] * 100:.2f}%")
df

  0%|          | 0/10000000 [00:00<?, ?it/s]

% of duplicate names: 9.56%
% of duplicate emails: 7.51%


,name,email,address
0,Larry Klein74,blake17@example.net37,"PSC 0274, Box 6368\nAPO AP 99086"
1,James Cook153,asmith@example.com76,"320 Kristin Parks\nDeniseshire, PA 79279"
2,Jeffery Hudson37,thompsonjonathan@example.net18,Unit 4883 Box 6026\nDPO AE 00823
3,Billy Moran263,kaylagarcia@example.net131,USNS Dawson\nFPO AP 47081
4,John Estrada18,vanessabuckley@example.com9,"0173 Amanda Roads\nDarlenehaven, NM 51077"
...,...,...,...
9999995,Jennifer Potter64,angela86@example.org32,"1048 Barry Bypass\nNorth Deborah, IN 29911"
9999996,Debra Nguyen48,johnsonjennifer@example.org24,"2075 Curry Wall\nWest Katrina, NJ 28813"
9999997,Sylvia Moore217,owensvanessa@example.net108,"5556 Noah Shoals Suite 563\nSylviaville, MD 06369"
9999998,Anthony Kent79,hunterlong@example.org39,"23124 Lisa Coves\nNorth Tracy, GU 61652"


In [28]:
print(f"Total duplicated names: {df['name'].value_counts().loc[lambda x: x > 1].shape[0]:,}")

Total duplicated names: 956,063


In [29]:
print(f"Total duplicated emails: {df['email'].value_counts().loc[lambda x: x > 1].shape[0]:,}")

Total duplicated emails: 750,725


In [30]:
non_unic_emails = df['email'].value_counts().loc[lambda x: x > 1].index
non_unic_emails

Index(['osmith@example.net166', 'zsmith@example.org97',
       'nsmith@example.com103', 'nsmith@example.org144',
       'esmith@example.com24', 'zsmith@example.org3', 'qsmith@example.org21',
       'fsmith@example.com31', 'fsmith@example.com52', 'esmith@example.net33',
       ...
       'jaredbrown@example.org88', 'bhuff@example.org99',
       'eric46@example.org40', 'matthew50@example.com161',
       'ymiller@example.net55', 'johnharris@example.org46',
       'crystal02@example.org51', 'christopher85@example.org156',
       'deborah91@example.com13', 'imartin@example.org144'],
      dtype='object', name='email', length=750725)

In [35]:
import recordlinkage

# In case we want to estimate the time to compute by chunks
# sample_df = df.sample(frac=0.1)

sample_df = df

time_start = pd.Timestamp.now()
rules = [
    {"field": "name", "type": "Exact", "weight": 50},
    {"field": "email", "type": "Exact", "weight": 20},
    {"field": "address", "type": "Fuzzy", "weight": 30}
]

comp = recordlinkage.Compare()

comp.exact('name', 'name', agree_value=50)
comp.exact('email', 'email', agree_value=20)
comp.string('address', 'address', method='levenshtein', threshold=0.85)

indexer = recordlinkage.Index()
indexer.block(left_on='name', right_on='name')
indexer.block(left_on='email', right_on='email')
candidate_pairs = indexer.index(sample_df)
time_end = pd.Timestamp.now()
print(f"Time to compute candidate pairs: {time_end - time_start}")
print(f"Number of candidate pairs: {candidate_pairs.shape[0]:,}")

Time to compute candidate pairs: 0 days 00:00:17.612178
Number of candidate pairs: 3,651,777


In [36]:
time_start = pd.Timestamp.now()
computed = comp.compute(candidate_pairs, df)
computed.columns = ['name_score', 'email_score', 'address_score']
computed['address_score_converted'] = computed['address_score'] * rules[2]['weight']
computed['score'] = computed[['name_score', 'email_score', 'address_score_converted']].sum(axis=1)
computed = computed.sort_values(by='score', ascending=False)
computed = computed.reset_index()
time_end = pd.Timestamp.now()
print(f"Time to compute scores: {time_end - time_start}")
print(f"Number of computed pairs: {computed.shape[0]:,}")

Time to compute scores: 0 days 00:00:55.267969
Number of computed pairs: 3,651,777


Prepare the final report

In [37]:
computed_grouped = (computed.groupby(['level_0', 'score'])
                    .agg(cluster_size = ('level_1', 'count'))
                    .reset_index())
computed_grouped.columns = ['Cluster Id', 'Score', 'Cluster size']
computed_grouped = computed_grouped[['Cluster Id', 'Cluster size', 'Score']]
computed_grouped

,Cluster Id,Cluster size,Score
0,3870,1,50.0
1,4089,1,50.0
2,4243,1,20.0
3,4615,1,50.0
4,4862,1,50.0
...,...,...,...
2385033,9999987,3,50.0
2385034,9999990,1,20.0
2385035,9999992,2,20.0
2385036,9999992,1,50.0


### What to do next? 

There are several strategies to consider. 

Strategy 1: For the cases when there is much more the 5-10% of duplicated names, or other exact columns, we can consider to prefilter the daraset to remove the duplicated records before the linkage process.

Strategy 2: Split dataset into **sorted** blocks and compare records only within the same block. This will reduce the number of comparisons and speed up the process. This strategy also adds the possibility to scale the process.

<br>

### Extreme cases

For the cases with huge datasets, we can consider to use approximate algorithms. These algorithms are able to find the similar records with high probability, but they are not able to find the exact matches. But this is the topic for another discussion.